In [1]:
import os
import numpy as np
import pandas as pd

import plotly
import plotly.graph_objs as go
#import matplotlib.pyplot as plt

#from tqdm import tqdm_notebook

In [2]:
plotly.offline.init_notebook_mode(connected=True)
pd.options.display.max_rows = 1000

In [3]:
def plot_signal(df, signal_name, stop_legend = False, opacity_cndl = 0.8):
    # рисуем свечной график с объемами
    # signal_name - колонка с сигналом
    # ниже кода для трех состояний сигнала: +1, -1, 0
    # +1 синий, -1 розовый, 0 цвет свечек не меняется


    color_signal_long = '#0028FF' # цвет сигнала для лонга - синий
    color_signal_short = '#ff00f1' # цвет сигнала для шорта - розовый
    
    mask_long = df[signal_name]>0
    mask_short = df[signal_name]<0
        
    name_long = 'Long_' + str(round(df[mask_long].shape[0]*100/df.shape[0],1)) + '%'
    name_short = 'Short_' + str(round(df[mask_short].shape[0]*100/df.shape[0],1)) + '%'
        
    ticker_name = df.Ticker[0]
    data = [ dict(
        name = ticker_name + '_' + signal_name,
        type = 'candlestick',
        x = df['Date_2'],
        yaxis = 'y2',
        open = df['Open'],
        high = df['High'],
        low = df['Low'],
        close = df['Close'],
        opacity = opacity_cndl
    ),
        dict(
        name = name_long,
        type = 'candlestick',
        x = df['Date_2'][mask_long],
        yaxis = 'y2',
        open = df['Open'][mask_long],
        high = df['High'][mask_long],
        low = df['Low'][mask_long],
        close = df['Close'][mask_long],
        hoverinfo='none',
        increasing = dict(line = dict(color = '#0028FF')),
        decreasing = dict(line = dict(color = '#0028FF'))
    ),
        dict(
        name = name_short,
        type = 'candlestick',
        x = df['Date_2'][mask_short],
        yaxis = 'y2',
        open = df['Open'][mask_short],
        high = df['High'][mask_short],
        low = df['Low'][mask_short],
        close = df['Close'][mask_short],
        hoverinfo='none',
        increasing = dict(line = dict(color = '#ff00f1')),
        decreasing = dict(line = dict(color = '#ff00f1'))
    )]

    layout=dict()

    fig = dict(data=data, layout=layout)

    fig['layout'] = dict()
    #fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
    #categoryorder = 'category ascending', 
    fig['layout']['xaxis'] = dict(type ='category', 
                                  tickfont=dict(size=10),
                                  rangeselector = dict(visible = True), 
                                  rangeslider = dict(visible = False))
    fig['layout']['yaxis'] = dict(domain = [0, 0.2], 
                                  showticklabels = False, 
                                  autorange = True, 
                                  type = 'log',
#                                  title = 'Volume'
                                 )
    fig['layout']['yaxis2'] = dict(domain = [0.1, 0.95], 
                                   autorange = True, 
                                   type = 'log',
                                   title = ticker_name + ' prices')
    fig['layout']['legend'] = dict(orientation = 'h', 
                                   y = 0.95, x = 0.1, 
                                   yanchor = 'bottom')
    fig['layout']['margin'] = dict(t = 40, b = 40, r = 40, l = 40)
    
    if stop_legend:
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sl_long'], type='scatter', yaxis='y2', 
                                line = dict(width = 1), marker=dict(color='#FF4500'), name='SL_long'))
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sl_short'], type='scatter', yaxis='y2', 
                            line = dict(width = 1), marker=dict(color='#FF4500'), name='SL_short'))
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sp_long'], type='scatter', yaxis='y2', 
                                line = dict(width = 1), marker=dict(color='#008B8B'), name='SP_long'))
        fig['data'].append(dict(x=df['Date_2'], y=df['ATR_sp_short'], type='scatter', yaxis='y2', 
                            line = dict(width = 1), marker=dict(color='#008B8B'), name='SP_short'))


    plotly.offline.iplot(fig)
    return

In [8]:
# переменные 
path = r'C:\Users\parsh\Desktop\Exchange\data\US_stocks_csv'
file_name = '/NYSE/ABX.csv'
file_ticker = path + file_name

# читаем базу, строим колонку Data2 для графиков

df = pd.read_csv(file_ticker)
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='ignore')
df['Date_2'] = df['Date'].dt.strftime('%d-%m-%Y')

# Turtle simple soup

In [9]:
extremum_window = 20 # баров для расчета колонки с n-барным экстремумом
col_max_name = 'max_'+str(extremum_window)+'_bars'
col_min_name = 'min_'+str(extremum_window)+'_bars'

# число дней с текущего, когда сравниваем - пробили n-барный экстремум или нет
# не путать с extremum_window
break_window = 20 

# зона "баров назад" (шифта) от текущего, в которую не должен попасть n-дневный экструмум
# из колонки col_max_name. На самом деле получается зона = (empty_zone-1), 
# то есть в дне shift(empty_zone) экстремум еще может быть 
empty_zone = 4

signal_col = 'SOUP_' + str(extremum_window) + '_' + str(break_window) + '_' + str(empty_zone)

In [10]:
# !!!!!!!!!!! Важно!!!!!!!!!!!
# nan присваиваем минус -1e9 - потому что не для акций, а индексов могут быть отрицательные значения!!!
# Следовательно когда ищем минимумы - лучше присваивать +1e9 (цены акций из-за adjusted 
# могут быть сильно выше разумных величин)

df[col_max_name] = df['High'][df['High'].rolling(extremum_window).max() == df['High']]
df[col_max_name] = df[col_max_name].fillna(-1e9)

df[col_min_name] = df['Low'][df['Low'].rolling(extremum_window).min() == df['Low']]
df[col_min_name] = df[col_min_name].fillna(1e9)

In [ ]:
df[col_max_name] = df['High'][df['High'].rolling(extremum_window).max() == df['High']]
df[col_max_name] = df[col_max_name].fillna(-1e9)

df['max_Nmax'] = df[col_max_name].shift(empty_zone).rolling(break_window-empty_zone+1).max()
df['max_high_empty_zone'] = df['High'].shift(1).rolling(empty_zone-1).max()
df['short_signal'] = ((df['High'] > df['max_Nmax']) & \
                        (df['max_Nmax'] > df['max_high_empty_zone']) & \
                        (df['Close'] < df['max_Nmax'])) * (-1)

In [11]:
# для шорта

df['max_Nmax'] = df[col_max_name].shift(empty_zone).rolling(break_window-empty_zone+1).max()
df['max_high_empty_zone'] = df['High'].shift(1).rolling(empty_zone-1).max()
df['short_signal'] = ((df['High'] > df['max_Nmax']) & \
                        (df['max_Nmax'] > df['max_high_empty_zone']) & \
                        (df['Close'] < df['max_Nmax'])) * (-1)

# для лонга

df['min_Nmin'] = df[col_min_name].shift(empty_zone).rolling(break_window-empty_zone+1).min()
df['min_low_empty_zone'] = df['Low'].shift(1).rolling(empty_zone-1).min()
df['long_signal'] = ((df['Low'] < df['min_Nmin']) & \
                        (df['min_Nmin'] < df['min_low_empty_zone']) & \
                        (df['Close'] > df['min_Nmin'])) * (1)

df[signal_col] = df['short_signal'] + df['long_signal']
df

,Ticker,Date,Open,High,Low,Close,Volume,Date_2,max_20_bars,min_20_bars,max_Nmax,max_high_empty_zone,short_signal,min_Nmin,min_low_empty_zone,long_signal,SOUP_20_20_4
0,ABX,1970-01-01 00:00:00.019850201,0.62448,0.62448,0.62448,0.62448,73341,01-01-1970,-1.000000e+09,1.000000e+09,NaN,NaN,0,NaN,NaN,0,0
1,ABX,1970-01-01 00:00:00.019850204,0.62448,0.62448,0.62448,0.62448,50762,01-01-1970,-1.000000e+09,1.000000e+09,NaN,NaN,0,NaN,NaN,0,0
2,ABX,1970-01-01 00:00:00.019850205,0.62448,0.62448,0.62448,0.62448,18415,01-01-1970,-1.000000e+09,1.000000e+09,NaN,NaN,0,NaN,NaN,0,0
3,ABX,1970-01-01 00:00:00.019850206,0.70254,0.70254,0.62448,0.62448,65334,01-01-1970,-1.000000e+09,1.000000e+09,NaN,0.62448,0,NaN,0.62448,0,0
4,ABX,1970-01-01 00:00:00.019850207,0.62448,0.62448,0.62448,0.62448,39873,01-01-1970,-1.000000e+09,1.000000e+09,NaN,0.70254,0,NaN,0.62448,0,0
5,ABX,1970-01-01 00:00:00.019850208,0.62448,0.62448,0.62448,0.62448,25541,01-01-1970,-1.000000e+09,1.000000e+09,NaN,0.70254,0,NaN,0.62448,0,0
6,ABX,1970-01-01 00:00:00.019850211,0.70254,0.70254,0.62448,0.62448,29304,01-01-1970,-1.000000e+09,1.000000e+09,NaN,0.70254,0,NaN,0.62448,0,0
7,ABX,1970-01-01 00:00:00.019850212,0.62448,0.62448,0.62448,0.62448,34749,01-01-1970,-1.000000e+09,1.000000e+09,NaN,0.70254,0,NaN,0.62448,0,0
8,ABX,1970-01-01 00:00:00.019850213,0.62448,0.62448,0.62448,0.62448,19696,01-01-1970,-1.000000e+09,1.000000e+09,NaN,0.70254,0,NaN,0.62448,0,0
9,ABX,1970-01-01 00:00:00.019850214,0.62448,0.70254,0.62448,0.70254,65975,01-01-1970,-1.000000e+09,1.000000e+09,NaN,0.70254,0,NaN,0.62448,0,0


In [14]:
# рисуем график
plot_signal(df, signal_name = signal_col, stop_legend = False, opacity_cndl = 0.3)